In [ ]:
!pip install pillow transformers > /dev/null 2>&1
!pip install --upgrade torch torchvision torchaudio > /dev/null 2>&1

In [ ]:
!python -c "import torch; print(torch.__version__)"

In [ ]:
import requests
import torch
from PIL import Image
from torch.nn.functional import cosine_similarity
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoProcessor

import os
from pathlib import Path

import hydra
from hydra.utils import instantiate
import numpy as np
import matplotlib.pyplot as plt

from navsim.common.dataloader import SceneLoader
from navsim.common.dataclasses import SceneFilter, SensorConfig, Scene, Camera

torch.cuda.is_available()

In [ ]:
import hydra
from hydra.utils import instantiate
from navsim.common.dataloader import SceneLoader
from navsim.common.dataclasses import SceneFilter, SensorConfig, Scene
from pathlib import Path
import numpy as np
from PIL import Image # Make sure PIL (Pillow) is imported if you haven't already

SPLIT = "mini"  # ["mini", "test", "trainval"]
FILTER = "all_scenes" # Or your specific filter if needed

hydra.initialize(config_path="../navsim/navsim/planning/script/config/common/train_test_split/scene_filter", version_base=None) # Add version_base=None to silence hydra warning
cfg = hydra.compose(config_name=FILTER)
scene_filter: SceneFilter = instantiate(cfg)
openscene_data_root = Path("../dataset")

# Define the base paths correctly based on your directory structure
navsim_log_path = openscene_data_root / f"{SPLIT}_navsim_logs" / SPLIT
sensor_blob_path = openscene_data_root / f"{SPLIT}_sensor_blobs" / "sensor_blobs" / SPLIT

# Optional: Check if paths exist before creating SceneLoader
if not navsim_log_path.exists():
    raise FileNotFoundError(f"Navsim log path not found: {navsim_log_path}")
if not sensor_blob_path.exists():
     raise FileNotFoundError(f"Sensor blob path not found: {sensor_blob_path}")


In [ ]:

print(f"Using Navsim log path: {navsim_log_path}")
print(f"Using Sensor blob path: {sensor_blob_path}")

scene_loader = SceneLoader(
    data_path=navsim_log_path,
    original_sensor_path=sensor_blob_path,
    scene_filter=scene_filter,
    # Check if these synthetic paths are needed and exist correctly.
    # They might also have incorrect base paths if you're using them.
    # Setting to None if not used/downloaded.
    synthetic_sensor_path=None, # openscene_data_root / "warmup_two_stage/sensor_blobs",
    synthetic_scenes_path=None, # openscene_data_root / "warmup_two_stage/synthetic_scene_pickles",
    sensor_config=SensorConfig.build_all_sensors(),
)

print(f"Loaded {len(scene_loader)} scenes from {len(scene_loader.tokens)} tokens.")



In [ ]:

# --- Now try loading and viewing a scene ---
if scene_loader.tokens: # Check if any tokens were loaded
    token = np.random.choice(scene_loader.tokens)
    print(f"Attempting to load scene for token: {token}")
    scene = scene_loader.get_scene_from_token(token)
    print(f"Successfully loaded scene for token: {token}")

    # Example: Access and display the front camera image of the current frame (frame_idx=0)
    if scene.frames and scene.frames[0].cameras: # Check if frame and its cameras object exist
        current_frame = scene.frames[0]
        cameras_object = current_frame.cameras # Get the Cameras object

        # Access the 'cam_f0' attribute directly from the Cameras object
        front_camera_data: Camera = cameras_object.cam_f0

        # Check if the image attribute within the Camera object is not None
        if front_camera_data.image is not None:
            print("Front camera image loaded successfully.")
            # Display the image
            plt.figure(figsize=(10, 6)) # Optional: Adjust figure size
            plt.imshow(front_camera_data.image)
            plt.title(f"Front Camera (CAM_F0) - Token: {token}")
            plt.axis('off') # Hide axes
            plt.show()
        else:
            # This case occurs if SensorConfig excluded cam_f0 or data was missing
            print("Front camera (cam_f0) image data is None.")
    else:
        print("Could not access frame or cameras data for the current frame.")